In [3]:
from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
from os.path import exists
import shutil
import sys
import time

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
    
#From my EEG package 
import run_expts
import constants


#Let me see as many results as I want to see
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 20)

In [21]:
#This path also has Wake data in it
load_path_2 = '/export/sphere/ebirah/ko20929/RBD_using_custom_package/Execute New Experiments/Catch_22_features/'

#This path only has N1 and REM
load_path = '/export/sphere/ebirah/ko20929/RBD_using_custom_package/Data/Catch_22_features/'

In [24]:
X = pd.read_hdf(load_path_2 + 'Wake_c_22_feautures.h5', key='df', mode='r')

X

,Fp1_DN_HistogramMode_5,Fp1_DN_HistogramMode_10,Fp1_SB_BinaryStats_diff_longstretch0,Fp1_DN_OutlierInclude_p_001_mdrmd,Fp1_DN_OutlierInclude_n_001_mdrmd,Fp1_CO_f1ecac,Fp1_CO_FirstMin_ac,Fp1_SP_Summaries_welch_rect_area_5_1,Fp1_SP_Summaries_welch_rect_centroid,Fp1_FC_LocalSimple_mean3_stderr,...,Oz_SB_BinaryStats_mean_longstretch1,Oz_SB_MotifThree_quantile_hh,Oz_FC_LocalSimple_mean1_tauresrat,Oz_CO_Embed2_Dist_tau_d_expfit_meandiff,Oz_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,Oz_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,Oz_SB_TransitionMatrix_3ac_sumdiagcov,Oz_PD_PeriodicityWang_th0_01,Oz_StandardDeviation,Oz_Mean
0,6.431152,1.981088,79.0,-0.000922,-0.000043,6.0,16.0,63.775317,0.188009,7.022472,...,13.0,1.947563,0.111111,0.019322,0.46,0.24,0.000243,29.0,8.920532,0.005542
1,5.847820,2.878111,107.0,-0.043178,-0.044650,8.0,18.0,31.813945,0.171614,2.524346,...,24.0,1.662810,0.272727,0.065149,0.46,0.24,0.000195,35.0,6.614500,-0.001662
2,-23.120010,-12.296121,284.0,-0.030010,-0.139355,29.0,122.0,530.420328,0.032885,5.888387,...,23.0,1.557094,0.102564,0.045291,0.44,0.48,0.000152,25.0,10.656027,0.011851
3,14.227444,4.553551,204.0,0.081484,0.099961,37.0,14.0,222.429637,0.020133,4.486776,...,20.0,1.681230,0.375000,0.041792,0.54,0.18,0.000293,25.0,8.777325,0.004510
4,-1.094439,5.852149,169.0,0.107227,0.072331,31.0,16.0,76.085271,0.049471,4.402563,...,18.0,1.894260,0.666667,0.009321,0.52,0.54,0.000145,6.0,11.135686,0.009649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,5.696133,2.390115,159.0,-0.149564,-0.124264,6.0,14.0,47.237489,0.235993,3.932522,...,21.0,1.726625,0.428571,0.042341,0.46,0.20,0.000087,25.0,9.399973,-0.000901
52,-0.609127,2.654985,95.0,0.091518,0.084738,5.0,5.0,42.100568,0.239349,4.780232,...,20.0,1.780693,0.375000,0.032510,0.48,0.50,0.000167,24.0,9.467021,0.000678
53,3.729443,-0.829666,152.0,-0.190927,-0.143537,7.0,15.0,72.617439,0.197884,4.308705,...,21.0,1.642674,0.571429,0.039004,0.48,0.20,0.000287,26.0,9.344512,0.001569
54,5.556511,-9.560163,185.0,0.231823,0.201816,4.0,4.0,93.754766,0.068358,8.514457,...,20.0,1.799573,0.200000,0.026589,0.42,0.44,0.000180,6.0,9.581137,0.020620


In [18]:
X = pd.read_hdf(load_path + 'N1_c_22_feautures.h5', key='df', mode='r')
y = pd.read_hdf(load_path + 'REM_y.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + 'REM_groups.h5', key='df', mode='r')

#1.Generate expt specific X,y,groups
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, 1 )

# # #One channel per region is worse than three channels 
# X_cols = X_expt.columns
# one_chan_per_reg = [channels[1] for channels in reg_chs]
# selected_cols = [col for col in X_cols if any(col.startswith(prefix) for prefix in one_chan_per_reg)]
# X_expt = X_expt[selected_cols]

X_expt = X_expt[ [col for col in X_expt.columns if 'Fpz' == col[:3] or 'FCz'== col[:3] or 'POz'== col[:3] ] ]

# X_expt = X_expt[ [col for col in X_expt.columns if chan == col[:len(chan)] for chan in channels ]]

In [16]:
X_2 = pd.read_hdf(load_path_2 + 'N1_c_22_feautures.h5', key='df', mode='r')
X_2

,Fp1_DN_HistogramMode_5,Fp1_DN_HistogramMode_10,Fp1_SB_BinaryStats_diff_longstretch0,Fp1_DN_OutlierInclude_p_001_mdrmd,Fp1_DN_OutlierInclude_n_001_mdrmd,Fp1_CO_f1ecac,Fp1_CO_FirstMin_ac,Fp1_SP_Summaries_welch_rect_area_5_1,Fp1_SP_Summaries_welch_rect_centroid,Fp1_FC_LocalSimple_mean3_stderr,...,Oz_SB_BinaryStats_mean_longstretch1,Oz_SB_MotifThree_quantile_hh,Oz_FC_LocalSimple_mean1_tauresrat,Oz_CO_Embed2_Dist_tau_d_expfit_meandiff,Oz_SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,Oz_SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,Oz_SB_TransitionMatrix_3ac_sumdiagcov,Oz_PD_PeriodicityWang_th0_01,Oz_StandardDeviation,Oz_Mean
0,0.442715,-3.487699,418.0,-0.243845,-0.191495,56.0,23.0,66.166775,0.020900,3.180520,...,26.0,1.563868,0.024194,0.053283,0.54,0.60,0.000392,33.0,8.051308,0.018304
1,0.571947,-5.833110,817.0,-0.175102,-0.224894,99.0,16.0,76.208986,0.011792,3.367519,...,27.0,1.557703,0.009804,0.045082,0.58,0.62,0.000446,15.0,6.583402,0.080150
2,0.475425,-2.521421,311.0,-0.062295,-0.067334,12.0,18.0,20.173259,0.063900,1.627164,...,34.0,1.427122,0.038674,0.053903,0.48,0.54,0.000472,33.0,6.197134,-0.021716
3,-2.558197,3.309954,526.0,-0.189136,-0.240617,38.0,18.0,57.825156,0.035857,2.660376,...,33.0,1.447576,0.105263,0.026107,0.56,0.56,0.000199,32.0,10.330993,0.002295
4,3.416201,-1.270598,237.0,0.088325,0.052062,17.0,18.0,73.393508,0.040363,2.558762,...,42.0,1.407341,0.073394,0.019118,0.62,0.64,0.000262,32.0,14.199590,0.079316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,-3.834144,0.885067,620.0,-0.674889,-0.124165,42.0,17.0,52.095342,0.024574,2.237312,...,40.0,1.406062,0.036036,0.022965,0.40,0.44,0.000164,28.0,9.753643,-0.017608
67,-0.856291,2.697464,529.0,-0.165081,-0.144093,58.0,17.0,36.568389,0.022171,2.385022,...,31.0,1.532185,0.021429,0.070247,0.46,0.52,0.000329,21.0,7.476408,-0.029247
68,2.469949,-2.945690,379.0,-0.029857,-0.016726,13.0,15.0,24.715065,0.059346,2.571955,...,26.0,1.616849,0.016129,0.033109,0.50,0.54,0.000166,24.0,6.747228,-0.016754
69,-3.489416,1.827825,259.0,0.046185,0.021654,41.0,113.0,70.747111,0.026509,2.876240,...,32.0,1.451696,0.038835,0.026161,0.52,0.56,0.000941,726.0,9.007963,-0.036292


In [10]:
reg_chs=['Fp1', 'Fpz', 'Fp2'], ['F1', 'Fz', 'F2'], ['AF7', 'F7', 'F5', 'F3'], ['AF8', 'F8', 'F6', 'F4'], \
            ['FCz', 'FC1', 'FC2', 'Cz', 'C1', 'C2', 'CP1', 'CP2'], ['FC5', 'FC3', 'C5', 'C3', 'CP5', 'CP3'], \
            ['FC6', 'FC4', 'C6', 'C4', 'CP6', 'CP4'], ['TP7', 'T7', 'FT7', 'FT9'], ['TP8', 'T8', 'FT8', 'FT10'], \
            ['P1', 'P2', 'Pz'], ['P3', 'P5', 'P7', 'PO7'], ['P4', 'P6', 'P8', 'PO8'], ['POz', 'O1', 'O2', 'Oz']

In [11]:
one_chan_per_reg = [channels[1] for channels in reg_chs]
len(one_chan_per_reg)

13

In [13]:
acc_scores = []
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_expt , y_expt, test_size=0.2)
    clf = RandomForestClassifier()

    t1 = time.time()

    clf.fit(X_train, y_train)
    y_preds = clf.predict(X_test)
    
    acc = accuracy_score(y_preds , y_test )
    

    print(acc )
    
    acc_scores.append(acc)

    t2 = time.time()

mean_acc = np.mean(np.array(acc_scores))
print('mean accuracy is:' + str(mean_acc) )
#     print(t2-t1)

0.75
0.625
0.875
0.625
1.0
0.625
0.875
0.875
0.5
0.5
mean accuracy is:0.725


In [8]:
#Best mean is around 70 with features from three different channels